# Fatima Fellowship Quick Coding Challenge (Pick 1)

Thank you for applying to the Fatima Fellowship. To help us select the Fellows and assess your ability to do machine learning research, we are asking that you complete a short coding challenge. Please pick **1 of these 5** coding challenges, whichever is most aligned with your interests. 

**Due date: 1 week**

**How to submit**: Please make a copy of this colab notebook, add your code and results, and submit your colab notebook to the submission link below. If you have never used a colab notebook, [check out this video](https://www.youtube.com/watch?v=i-HnvsehuSw).

**Submission link**: https://airtable.com/shrXy3QKSsO2yALd3

# 1. Deep Learning for Vision

**Upside down detector**: Train a model to detect if images are upside down

* Pick a dataset of natural images (we suggest looking at datasets on the [Hugging Face Hub](https://huggingface.co/datasets?task_categories=task_categories:image-classification&sort=downloads))
* Synthetically turn some of images upside down. Create a training and test set.
* Build a neural network (using Tensorflow, PyTorch, or any framework you like)
* Train it to classify image orientation until a reasonable accuracy is reached
* [Upload the the model to the Hugging Face Hub](https://huggingface.co/docs/hub/adding-a-model), and add a link to your model below.
* Look at some of the images that were classified incorrectly. Please explain what you might do to improve your model's performance on these images in the future (you do not need to impelement these suggestions)

**Submission instructions**: Please write your code below and include some examples of images that were classified

###Pick a dataset of natural images (we suggest looking at datasets on the Hugging Face Hub)

In [ ]:
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("mnist")

In [ ]:
# imports

import numpy as np
import math
import pandas as pd

import torch
import torchvision.transforms as transforms

###Synthetically turn some of images upside down. Create a training and test set.

In [ ]:
# from sklearn.model_selection import train_test_split

In [ ]:
# functions to make train and test dataset

dim = np.array(dataset['train'][0]["image"]).shape[0]
channel_count = 1
img_str = "image"

def convert_img_to_tensor(image):
  transform = transforms.Compose([transforms.PILToTensor()])

  img_tensor = transform(image).reshape(1,dim,dim,channel_count)

  img_tensor = np.pad(img_tensor, ((0,0), (2,2), (2,2), (0,0)))

  # print(img_tensor.shape)
  return img_tensor

def get_tensor_array(mydf):
  arr_orig = np.array([])

  for i in range(len(mydf)):
    image = mydf[img_str][i]
    img_tensor = convert_img_to_tensor(image)
    if i == 0:
      arr_orig = np.array(img_tensor)
    else:
      arr_orig = np.concatenate((arr_orig, img_tensor))

  return arr_orig


def do_preprocessing(mydf, data_count, if_train):
  mydf_X_orig = mydf.copy()

  # if if_train == 1:
  #   # keeping the ratio of the numbers same in the training dataset
  #   y = mydf_X_orig["label"]
  #   X = mydf_X_orig.drop('label', axis=1)
  #   # X_train, X_test, y_train, y_test
  #   non_used_X_train, used_X_train, non_used_y_train, used_y_train = train_test_split(X, y, stratify=y, test_size=0.45) # 0.08
  #   mydf_X_orig = used_X_train
  # else:
  #   mydf_X_orig = mydf_X_orig.loc[:(data_count-1)]
  #   mydf_X_orig = mydf_X_orig.drop('label', axis=1)

  mydf_X_orig = mydf_X_orig.drop('label', axis=1)
  
  mydf_X_orig = mydf_X_orig.reset_index()
  mydf_X_orig = mydf_X_orig.drop("index", axis = 1)

  arr_X_orig = get_tensor_array(mydf_X_orig)
  arr_Y_orig = np.zeros(len(arr_X_orig))

  # print("arr_X_orig.shape: " + str(arr_X_orig.shape))
  # print("arr_Y_orig.shape: " + str(arr_Y_orig.shape))

  mydf_X_rot_list = []

  for i in range(len(mydf_X_orig)):
    image = mydf_X_orig[img_str][i]
    rot_img = image.rotate(angle=180)
    mydf_X_rot_list.append(rot_img)

  # print(len(mydf_X_rot_list))

  mydf_X_rot = pd.DataFrame(mydf_X_rot_list, columns=[img_str])
  # print("mydf_X_rot.shape: " + str(mydf_X_rot.shape))

  arr_X_rot = get_tensor_array(mydf_X_rot)
  arr_Y_rot = np.ones(len(arr_X_rot))

  # print("arr_X_rot.shape: " + str(arr_X_rot.shape))
  # print("arr_Y_rot.shape: " + str(arr_Y_rot.shape))

  return arr_X_orig, arr_Y_orig, arr_X_rot, arr_Y_rot

def do_shuffling(my_arr_X, my_arr_y):
  index_array = np.arange(my_arr_X.shape[0])
  sample_index = np.random.choice(index_array, my_arr_X.shape[0], replace=False)

  my_arr_X = my_arr_X[sample_index]
  my_arr_y = my_arr_y[sample_index]

  return my_arr_X, my_arr_y

In [ ]:
train_df_X_orig = pd.DataFrame(dataset['train'])
test_df_X_orig = pd.DataFrame(dataset['test'])

# deleting rows with symmetric numbers from the dataset 
delete_row = [0, 1, 3, 6, 8, 9]

for i in range(len(delete_row)):
  train_df_X_orig = train_df_X_orig[train_df_X_orig.label != delete_row[i]]
  test_df_X_orig = test_df_X_orig[test_df_X_orig.label != delete_row[i]]

# to keep the track of the indices of the test dataset; later we will need it to find out which image is classified incorrectly
test_index_list = list(test_df_X_orig.index)

train_df_X_orig = train_df_X_orig.reset_index()
test_df_X_orig = test_df_X_orig.reset_index()

train_df_X_orig = train_df_X_orig.drop("index", axis = 1)
test_df_X_orig = test_df_X_orig.drop("index", axis = 1)

print("before preprocessing:")
print("train_df_X_orig.shape: " + str(train_df_X_orig.shape))
print("test_df_X_orig.shape: " + str(test_df_X_orig.shape))

before preprocessing:
train_df_X_orig.shape: (23486, 2)
test_df_X_orig.shape: (3934, 2)


In [ ]:
# make train set
train_arr_X_orig, train_arr_y_orig, train_arr_X_rot, train_arr_y_rot = do_preprocessing(train_df_X_orig, 1000, 1)

train_X = np.concatenate((train_arr_X_orig, train_arr_X_rot))
train_y = np.concatenate((train_arr_y_orig, train_arr_y_rot))

train_X, train_y = do_shuffling(train_X, train_y)

# make test set
test_arr_X_orig, test_arr_y_orig, test_arr_X_rot, test_arr_y_rot = do_preprocessing(test_df_X_orig, 500, 0)

test_X = np.concatenate((test_arr_X_orig, test_arr_X_rot))
test_y = np.concatenate((test_arr_y_orig, test_arr_y_rot))

print("after preprocessing:")
print("train_X.shape: " + str(train_X.shape))
print("train_Y.shape: " + str(train_y.shape))
print("test_X.shape: " + str(test_X.shape))
print("test_Y.shape: " + str(test_y.shape))

after preprocessing:
train_X.shape: (46972, 32, 32, 1)
train_Y.shape: (46972,)
test_X.shape: (7868, 32, 32, 1)
test_Y.shape: (7868,)


In [ ]:
train_X = train_X.astype("float32") / 255
test_X = test_X.astype("float32") / 255

train_y = np.asarray(train_y).astype('float32').reshape((-1,1))
test_y = np.asarray(test_y).astype('float32').reshape((-1,1))

print("train_X.shape: " + str(train_X.shape))
print("train_Y.shape: " + str(train_y.shape))
print("test_X.shape: " + str(test_X.shape))
print("test_Y.shape: " + str(test_y.shape))

train_X.shape: (46972, 32, 32, 1)
train_Y.shape: (46972, 1)
test_X.shape: (7868, 32, 32, 1)
test_Y.shape: (7868, 1)


###Build a neural network (using Tensorflow, PyTorch, or any framework you like)

In [ ]:
height = 32
width = 32
channels = channel_count

# n_classes = 360
input_shape = (height, width, channels)

epochs = 50
batch_size = 32

In [ ]:
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping,CSVLogger,ReduceLROnPlateau,TensorBoard
from keras.models import Model

In [ ]:
vgg_model = tf.keras.applications.VGG19(
    include_top=False,
    # weights="imagenet",
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
)

base_model = vgg_model

import tensorflow.keras as K
model = K.models.Sequential()
model.add(base_model)
model.add(K.layers.Flatten())
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(256, activation='relu'))
model.add(K.layers.Dropout(0.2))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(128, activation='relu'))
model.add(K.layers.Dropout(0.2))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(64, activation='relu'))
model.add(K.layers.Dropout(0.2))
model.add(K.layers.BatchNormalization())
# model.add(K.layers.Dense(1, activation='sigmoid', name='fc1'))
model.add(K.layers.Dense(1, activation='sigmoid'))

print("added")

model.compile(loss='binary_crossentropy',
                  optimizer=K.optimizers.Adam(learning_rate=0.001),
                  metrics=['accuracy']
              )

print("compiled")

added
compiled


### Train it to classify image orientation until a reasonable accuracy is reached

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)

batch_size = 64
nb_epoch = 50
history = model.fit(train_X, train_y, 
                    batch_size=batch_size, epochs=nb_epoch,
                    callbacks=[early_stopping],
                    verbose=1, 
                    validation_split=0.2, 
                    # validation_data = (valid_X, valid_y),
                    shuffle=True
                    )

print("fitted")

Epoch 1/50
588/588 [==============================] - 79s 110ms/step - loss: 0.1407 - accuracy: 0.9319 - val_loss: 0.6540 - val_accuracy: 0.7750
Epoch 2/50
588/588 [==============================] - 63s 107ms/step - loss: 0.0402 - accuracy: 0.9856 - val_loss: 0.1781 - val_accuracy: 0.9256
Epoch 3/50
588/588 [==============================] - 64s 108ms/step - loss: 0.0293 - accuracy: 0.9901 - val_loss: 2.3166 - val_accuracy: 0.5069
Epoch 4/50
588/588 [==============================] - 64s 108ms/step - loss: 0.0225 - accuracy: 0.9926 - val_loss: 0.1270 - val_accuracy: 0.9613
Epoch 5/50
588/588 [==============================] - 64s 108ms/step - loss: 0.0206 - accuracy: 0.9934 - val_loss: 0.3757 - val_accuracy: 0.8899
Epoch 6/50
588/588 [==============================] - 64s 108ms/step - loss: 0.0196 - accuracy: 0.9938 - val_loss: 3.2893 - val_accuracy: 0.5355
Epoch 7/50
588/588 [==============================] - 64s 108ms/step - loss: 0.0198 - accuracy: 0.9937 - val_loss: 3.3238 - val_ac

In [ ]:
print(history.history.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [ ]:
import matplotlib.pyplot as plt

model_history = history
plt.figure(figsize=(18,8))

plt.suptitle('Loss and Accuracy Plots', fontsize=18)

plt.subplot(1,2,1)
plt.plot(model_history.history['loss'], label='Training Loss')
plt.plot(model_history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Number of epochs', fontsize=15)
plt.ylabel('Loss', fontsize=15)

plt.subplot(1,2,2)
plt.plot(model_history.history['accuracy'], label='Train Accuracy')
plt.plot(model_history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.xlabel('Number of epochs', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.show()

###Look at some of the images that were classified incorrectly. Please explain what you might do to improve your model's performance on these images in the future (you do not need to impelement these suggestions)

In [ ]:
# predict_labels = model.predict(test_X)
y_pred = (model.predict(test_X) > 0.5).astype("int32") 

In [ ]:
# count_0 = 0
# count_1 = 0

# print(len(test_y)/2)
# for i in range(len(test_y)):
#     if test_y[i] == y_pred[i]:
#         if test_y[i] == 0:
#             count_0 += 1
#         else:
#             count_1 += 1
            
# print(count_0)
# print(count_1)

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score

labels = ['0', '1']
print(classification_report(test_y, y_pred, target_names=labels))

              precision    recall  f1-score   support

           0       0.70      1.00      0.83      3934
           1       1.00      0.58      0.73      3934

    accuracy                           0.79      7868
   macro avg       0.85      0.79      0.78      7868
weighted avg       0.85      0.79      0.78      7868



### correctly classified image

In [ ]:
correct_i = -1
for i in range(len(test_y)):
  if test_y[i] == y_pred[i]:
    correct_i = i
    break

print(correct_i)

0


In [ ]:
check_image_index = test_index_list[correct_i]
check_image =  dataset['test'][check_image_index]['image']
check_image

### incorrectly classified image

In [ ]:
incorrect_i = -1

 # making a list of incorrectly classified images

incorrect_list = []
for i in range(len(test_y)):
  if test_y[i] != y_pred[i]:
    incorrect_list.append(i)
    # incorrect_i = i
    # break

# print(incorrect_i)

In [ ]:
# incorrectly classified image 1

half_len_test = int(len(test_y)/2)

incorrect_i = incorrect_list[0]
if incorrect_i >= half_len_test:
  incorrect_i = incorrect_i - half_len_test
check_image_index = test_index_list[incorrect_i]
check_image =  dataset['test'][check_image_index]['image']
check_image

In [ ]:
rot_check_img = check_image.rotate(angle=180)
rot_check_img

# similarity with 7

In [ ]:
# incorrectly classified image 2

half_len_test = int(len(test_y)/2)
incorrect_i = incorrect_list[1]
if incorrect_i >= half_len_test:
  incorrect_i = incorrect_i - half_len_test
check_image_index = test_index_list[incorrect_i]
check_image =  dataset['test'][check_image_index]['image']
check_image

In [ ]:
rot_check_img = check_image.rotate(angle=180)
rot_check_img

# similarity with 5

My train accuracy is 0.9951 and test accuracy is 0.79.
The validation curve is not smooth, hence there may be some overfitting problem. To improve the model, hypermeter tuning as well as cross validation can be used. Also curating some data by observing may also help to improve model learning.

**Write up**: 
* Link to the model on Hugging Face Hub: 
* Include some examples of misclassified images. Please explain what you might do to improve your model's performance on these images in the future (you do not need to impelement these suggestions)